In [25]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import glob
import re

In [39]:
files = glob.glob('images/*')
imageSizes = []

for f in files:
    m = re.search('images/(\d+).jpg', f)
    imageId = int(m.group(1))
    img = ndimage.imread(f)
    imageSizes.append([imageId, img.shape[0],img.shape[1]])
    
imageSizes = pd.DataFrame(imageSizes, columns=['id', 'width', 'height'])
imageSizes.set_index('id', inplace=True)
#     print img.shape
# img = ndimage.imread('images/6.jpg')
# plt.imshow(img, cmap='gray')
# plt.colorbar()

In [42]:
imageSizes.sort_values(by='width', ascending=False)

,width,height
id,,
573,1089,1261
957,1056,1254
124,1050,1261
1350,1043,1188
446,1023,1208
686,1023,845
691,1010,951
389,1004,1380
1476,1004,799
